# Imports

In [1]:
# import time
from technician_routing.routing_toolkit import RoutingConfig, RoutingInterface

# Demo

#### Configure and Load Data

In [2]:
config = RoutingConfig(
    'u:/data/darnley_tech_address.csv', 
    'u:/data/address_lat_long_121923.csv',
    40, # MPH
    55, # Minutes Per Service
    60*7 # Max Minutes per Route
)
interface = RoutingInterface(config).load_data()

Set 5269 client addresses
Using uniform minutes on location of 55
Data Loaded


#### Choose your clients

In [3]:
focus_addresses = interface.client_addresses[0:100]

#### Choose your technicians

In [4]:
#drivers = 25 # Weighted-Random Sample
drivers = [0,1,2,3,4,5,6] * 4 # Four of each technician

#### Optionally Ovewrite certain defaults at runtime

In [5]:
interface \
    .set_driver_speed(45) \
    .set_addresses(focus_addresses) \
    .set_time_on_location(65)

Using driver speed of 45 MPH
Set 100 client addresses
Using uniform minutes on location of 65


#### Run

In [6]:
# Random Drivers
routes = interface.run(route_drivers=drivers)
routes

[0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6]
Calculated 107 node durations in 0.021976947784423828
Running routing for 107 addresses and 28 drivers
Solved objective value 7312
Skipped unused route 0 starting at 0
Skipped unused route 7 starting at 7
Skipped unused route 8 starting at 8
Skipped unused route 9 starting at 9
Skipped unused route 15 starting at 15
Skipped unused route 21 starting at 21
Skipped unused route 22 starting at 22


[[1, 43, 72, 94, 86, 1],
 [2, 99, 19, 32, 27, 48, 2],
 [3, 71, 79, 106, 89, 10, 3],
 [4, 51, 20, 13, 53, 30, 4],
 [5, 52, 26, 37, 73, 41, 5],
 [6, 33, 28, 97, 31, 29, 6],
 [3, 102, 88, 101, 77, 21, 3],
 [4, 14, 16, 18, 25, 15, 4],
 [5, 12, 98, 75, 42, 57, 5],
 [6, 87, 39, 54, 47, 38, 6],
 [0, 76, 60, 0],
 [2, 8, 81, 22, 103, 46, 2],
 [3, 65, 23, 82, 66, 24, 3],
 [4, 68, 104, 74, 69, 35, 55, 4],
 [5, 62, 45, 67, 91, 93, 5],
 [6, 83, 84, 11, 70, 64, 7, 6],
 [2, 34, 49, 90, 56, 105, 2],
 [3, 100, 85, 63, 78, 3],
 [4, 50, 17, 96, 44, 95, 4],
 [5, 92, 36, 80, 5],
 [6, 40, 9, 58, 61, 59, 6]]

#### Detail Routes

In [9]:
for route_number, route in enumerate(routes):
    
    travel_time = 0
    for start,end in zip(route, route[1:]):
        travel_time += interface.solver.distances[start, end]

    site_time = 0
    for location in route[1:-1]:
        site_time += interface.solver.demands[location]

    tech = interface.df_techs.iloc[route[0], :]
    #print(tech)
     
    print(f'Route {route_number}: {int(travel_time + site_time)}','minutes')
    print(f'Technician {route[0]}: {tech["Tech"]} - {tech["address"]}')

    for stop in route[1:-1]:
        print(f'\t{stop}: {interface.last_addresses[stop]}')
    

    print(int(travel_time), 'travel minutes') 
    print(site_time, 'minutes on site')
    print('\n')

Route 0: 293 minutes
Technician 1: Michael Toia - 1164 Park Ave, Plainfield, NJ 07060, USA
	43: 1 Frost Ln, New Providence, NJ 07974, USA
	72: 1 Park Dr, Westfield, NJ 07090, USA
	94: 1 Tuscan Rd, Edison, NJ 08820, USA
	86: 1 S Louis St, Fords, NJ 08863, USA
33 travel minutes
260 minutes on site


Route 1: 369 minutes
Technician 2: Tranell Morton - 115 Spruce St, South Plainfield, NJ 07080, USA
	99: 1 Wieczorkowski Ave, Parlin, NJ 08859, USA
	19: 1 Barrier Dr, Hazlet, NJ 07730, USA
	32: 1 Crestwood Rd, Matawan, NJ 07747, USA
	27: 1 Cherry Creek Dr, Matawan, NJ 07747, USA
	48: 1 Harrison Ave, East Brunswick, NJ 08816, USA
44 travel minutes
325 minutes on site


Route 2: 363 minutes
Technician 3: Diego Costale - 221 Rutgers Rd, Piscataway, NJ 08854, USA
	71: 1 Oakmont Ave, East Brunswick, NJ 08816, USA
	79: 1 Rustic Dr, North Brunswick Township, NJ 08902, USA
	106: 10 Andover Dr, Kendall Park, NJ 08824, USA
	89: 1 Talcott Ct, Kendall Park, NJ 08824, USA
	10: 1 Andover Dr, Kendall Park, N